<a href="https://colab.research.google.com/github/Yug3737/Character-Recognition/blob/main/Letter_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision.datasets import EMNIST
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

emnist_data=EMNIST(root="./data",split="byclass",download=True)

print(emnist_data[0])
print(type(emnist_data[0]))

from collections import Counter
data_freq=Counter(labels)
data_freq


100%|██████████| 562M/562M [00:09<00:00, 60.5MB/s]


Extracting ./data/EMNIST/raw/gzip.zip to ./data/EMNIST/raw
(<PIL.Image.Image image mode=L size=28x28 at 0x7FAAF8505090>, 35)
<class 'tuple'>


In [2]:
# import cv2
# image = images[0]
# image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)
# cv2_imshow(images[0])

from collections import defaultdict
images = []
labels = []
labels_count=defaultdict(int)
for i in range(len(emnist_data)):
  if labels_count[emnist_data[i][1]]<4000:
    labels_count[emnist_data[i][1]]+=1
    images.append(emnist_data[i][0])
    labels.append(emnist_data[i][1])


In [3]:
len(labels)

217952

In [4]:
from collections import Counter
#labels_copy=labels
data_freq=Counter(labels)
data_freq

# Get min value and only keep that value

Counter({35: 2701,
         36: 4000,
         6: 4000,
         3: 4000,
         22: 4000,
         38: 2854,
         5: 4000,
         9: 4000,
         47: 4000,
         4: 4000,
         7: 4000,
         56: 2830,
         29: 4000,
         40: 4000,
         55: 4000,
         57: 2910,
         43: 4000,
         32: 4000,
         15: 4000,
         26: 2605,
         2: 4000,
         28: 4000,
         8: 4000,
         49: 4000,
         12: 4000,
         1: 4000,
         59: 2822,
         18: 4000,
         41: 2561,
         0: 4000,
         46: 2491,
         23: 4000,
         51: 2448,
         24: 4000,
         31: 4000,
         45: 1896,
         39: 4000,
         11: 3878,
         54: 2699,
         19: 3762,
         20: 2468,
         61: 2725,
         25: 4000,
         37: 4000,
         52: 2994,
         58: 2697,
         14: 4000,
         34: 4000,
         30: 4000,
         27: 4000,
         16: 2517,
         33: 2771,
         21: 4000,
   

In [5]:
print(min(labels))
print(max(labels))

0
61


In [6]:
filtered_images=[]
filtered_labels=[]

for i in range(len(labels)):
  if labels[i]>=10 and labels[i]<=61:
    filtered_images.append(images[i])
    filtered_labels.append(labels[i])

print(min(filtered_labels))
print(max(filtered_labels))

10
61


In [7]:
x=np.array(filtered_images)
# print(x[:5])
# print(y[:5])
y=np.array(filtered_labels)

print(x.shape)
print(y.shape)

(177952, 28, 28)
(177952,)
(177952, 28, 28)
(177952,)


In [8]:
x=x.reshape(x.shape[0],784)
y=np.eye(52)[y-10]

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)

print("Training Data Size:", len(x_train))
print("Testing data size:", len(x_test))


Training Data Size: 160156
Testing data size: 17796


In [10]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    # print("x = ", x)
    # print("np.max(x, axis=1, kdims=t) = ", np.max(x, axis=1, keepdims=True))
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability adjustment
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Neural Network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size):
        self.weights1 = np.random.randn(input_size, hidden_sizes[0]) * 0.01
        self.bias1 = np.zeros((1, hidden_sizes[0]))

        self.weights2 = np.random.randn(hidden_sizes[0], hidden_sizes[1]) * 0.01
        self.bias2 = np.zeros((1, hidden_sizes[1]))

        self.weights3 = np.random.randn(hidden_sizes[1], hidden_sizes[2]) * 0.01
        self.bias3 = np.zeros((1, hidden_sizes[2]))

        self.weights4 = np.random.randn(hidden_sizes[2], hidden_sizes[3]) * 0.01
        self.bias4 = np.zeros((1, hidden_sizes[3]))

        self.weights5 = np.random.randn(hidden_sizes[3], output_size) * 0.01
        self.bias5 = np.zeros((1, output_size))

        self.learning_rate = 0.001

    # Forward propagation
    def forward(self, X):
        self.z1 = np.dot(X, self.weights1) + self.bias1
        self.a1 = relu(self.z1)

        self.z2 = np.dot(self.a1, self.weights2) + self.bias2
        self.a2 = relu(self.z2)

        self.z3 = np.dot(self.a2, self.weights3) + self.bias3
        self.a3 = relu(self.z3)

        self.z4 = np.dot(self.a3, self.weights4) + self.bias4
        self.a4 = relu(self.z4)

        self.z5 = np.dot(self.a4, self.weights5) + self.bias5
        self.a5 = softmax(self.z5)

        return self.a5

    # Backpropagation
    def backward(self, X, y, y_pred):
        m = y.shape[0]  # Number of samples


        # Gradients for the output layer
        # print("ypred shape", y_pred.shape)
        # print("y shape", y.shape)
        d_z5 = y_pred - y

        d_weights5 = np.dot(self.a4.T, d_z5) / m
        d_bias5 = np.sum(d_z5, axis=0, keepdims=True) / m

        # Gradients for the 4th hidden layer
        d_a4 = np.dot(d_z5, self.weights5.T)
        d_z4 = d_a4 * relu_derivative(self.z4)
        d_weights4 = np.dot(self.a3.T, d_z4) / m
        d_bias4 = np.sum(d_z4, axis=0, keepdims=True) / m


        # Gradients for 3rd hidden layer
        d_a3 = np.dot(d_z4, self.weights4.T)
        d_z3 = d_a3 * relu_derivative(self.z3)
        d_weights3 = np.dot(self.a2.T, d_z3) / m
        d_bias3 = np.sum(d_z3, axis=0, keepdims=True) / m

        # Gradients for the second hidden layer
        d_a2 = np.dot(d_z3, self.weights3.T)
        d_z2 = d_a2 * relu_derivative(self.z2)
        d_weights2 = np.dot(self.a1.T, d_z2) / m
        d_bias2 = np.sum(d_z2, axis=0, keepdims=True) / m

        # Gradients for the first hidden layer
        d_a1 = np.dot(d_z2, self.weights2.T)
        d_z1 = d_a1 * relu_derivative(self.z1)
        d_weights1 = np.dot(X.T, d_z1) / m
        d_bias1 = np.sum(d_z1, axis=0, keepdims=True) / m

        # Update weights and biases
        self.weights5   -= self.learning_rate * d_weights5
        self.bias5      -= self.learning_rate * d_bias5
        self.weights4   -= self.learning_rate * d_weights4
        self.bias4      -= self.learning_rate * d_bias4
        self.weights3   -= self.learning_rate * d_weights3
        self.bias3      -= self.learning_rate * d_bias3
        self.weights2   -= self.learning_rate * d_weights2
        self.bias2      -= self.learning_rate * d_bias2
        self.weights1   -= self.learning_rate * d_weights1
        self.bias1      -= self.learning_rate * d_bias1

    # Training function
    def train(self, X, y, epochs, batch_size):
        for epoch in range(epochs):
            # Shuffle data
            indices = np.arange(X.shape[0])
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            # Mini-batch gradient descent
            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]

                # Forward and backward propagation
                y_pred = self.forward(X_batch)
                self.backward(X_batch, y_batch, y_pred)

            # Compute loss for the epoch
            y_pred = self.forward(X)
            # loss = -np.mean(np.sum(y * np.log(np.clip(y_pred, 1e-8, 1.0)), axis=1))  # Categorical cross-entropy
            loss = -np.mean(np.sum(y *np.log(y_pred + 1e-8), axis=1))
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    # Predict function
    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

    def predict_top_3(self, test_input):
      y_pred = self.forward(test_input)
      top_3_indices = np.argsort(y_pred, axis=1)[:, -3:][:, ::-1]  # Sort descending, get top 3
      top_3_probabilities = np.sort(y_pred, axis=1)[:, -3:][:, ::-1]

      result = []
      for i in range(top_3_indices.shape[0]):
        result.append({int(top_3_indices[i, j]): float(top_3_probabilities[i, j]) for j in range(3)})
      print(result)
      return result

# Initialize and train the network
input_size = 784  # Flattened image size
hidden_sizes = [512, 256, 128, 64]  # Number of neurons in hidden layers
output_size = 52 # Number of output classes
epochs = 50
batch_size = 64

nn = NeuralNetwork(input_size, hidden_sizes, output_size)
nn.train(x_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test data
y_test_pred = nn.predict(x_test)
y_test_true = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(y_test_pred == y_test_true)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/50, Loss: 3.9482
Epoch 2/50, Loss: 3.9399
Epoch 3/50, Loss: 3.8029
Epoch 4/50, Loss: 3.0404
Epoch 5/50, Loss: 1.9854
Epoch 6/50, Loss: 1.4618
Epoch 7/50, Loss: 1.2211
Epoch 8/50, Loss: 1.0606
Epoch 9/50, Loss: 0.9792
Epoch 10/50, Loss: 0.9062
Epoch 11/50, Loss: 0.8328
Epoch 12/50, Loss: 0.7764
Epoch 13/50, Loss: 0.7527
Epoch 14/50, Loss: 0.7473
Epoch 15/50, Loss: 0.7107
Epoch 16/50, Loss: 0.6776
Epoch 17/50, Loss: 0.6518
Epoch 18/50, Loss: 0.6332
Epoch 19/50, Loss: 0.5992
Epoch 20/50, Loss: 0.6037
Epoch 21/50, Loss: 0.6062
Epoch 22/50, Loss: 0.5827
Epoch 23/50, Loss: 0.5735
Epoch 24/50, Loss: 0.5443
Epoch 25/50, Loss: 0.5537
Epoch 26/50, Loss: 0.5236
Epoch 27/50, Loss: 0.5204
Epoch 28/50, Loss: 0.5457
Epoch 29/50, Loss: 0.5031
Epoch 30/50, Loss: 0.5589
Epoch 31/50, Loss: 0.5354
Epoch 32/50, Loss: 0.4934
Epoch 33/50, Loss: 0.4893
Epoch 34/50, Loss: 0.5596
Epoch 35/50, Loss: 0.4938
Epoch 36/50, Loss: 0.4705
Epoch 37/50, Loss: 0.5253
Epoch 38/50, Loss: 0.4545
Epoch 39/50, Loss: 0.

In [25]:
sample = x_test[29]
print(sample.shape)
nn.predict(x_test[29])
print("-----------")

nn.predict_top_3(x_test[0])

(784,)
-----------


AttributeError: 'NeuralNetwork' object has no attribute 'predict_top_3'

In [21]:
np.argmax(y_test[29])

33

In [23]:
import pickle
with open("letter_model.pkl", "wb") as file:
  pickle.dump(nn, file)